<a href="https://colab.research.google.com/github/MANOJHRMANOJHR/ai_assignments/blob/main/eight_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random, time, heapq, itertools
from collections import deque, defaultdict

# ---------- Puzzle model ----------
GOAL = (1,2,3,4,5,6,7,8,0)
MOVES = {0:(1,3),1:(0,2,4),2:(1,5),3:(0,4,6),4:(1,3,5,7),5:(2,4,8),6:(3,7),7:(4,6,8),8:(5,7)}

def is_solvable(state):
    arr = [x for x in state if x!=0]
    inv = sum(1 for i in range(len(arr)) for j in range(i+1,len(arr)) if arr[i]>arr[j])
    return inv % 2 == 0

def neighbors(state):
    z = state.index(0)
    for nb in MOVES[z]:
        s = list(state)
        s[z], s[nb] = s[nb], s[z]
        yield tuple(s), 1  # step cost 1

def random_start(max_shuffles=60):
    s = list(GOAL)
    z = s.index(0)
    for _ in range(random.randint(20,max_shuffles)):
        choices = MOVES[z]
        nb = random.choice(choices)
        s[z], s[nb] = s[nb], s[z]
        z = nb
    return tuple(s)

# ---------- Generic search framework ----------
class Result:
    def __init__(self, found, path_cost, nodes_expanded, depth, secs, path):
        self.found=found
        self.path_cost=path_cost
        self.nodes_expanded=nodes_expanded
        self.depth=depth
        self.secs=secs
        self.path=path

def reconstruct(parents, s):
    path=[s]
    while s in parents:
        s=parents[s]
        path.append(s)
    path.reverse()
    return path

def run_search(start, algo="bfs", heuristic=None):
    t0=time.perf_counter()
    nodes_expanded=0
    if start==GOAL:
        return Result(True,0,0,0,0.0,[start])

    if algo=="bfs":
        Q=deque([start])
        parents={}
        seen={start}
        while Q:
            s=Q.popleft()
            nodes_expanded+=1
            for nxt, c in neighbors(s):
                if nxt in seen: continue
                parents[nxt]=s
                seen.add(nxt)
                if nxt==GOAL:
                    path=reconstruct(parents,nxt)
                    return Result(True,len(path)-1,nodes_expanded,len(path)-1,time.perf_counter()-t0,path)
                Q.append(nxt)
        return Result(False, -1, nodes_expanded, -1, time.perf_counter()-t0, [])

    if algo=="dfs":
        stack=[start]
        parents={}
        seen={start}
        while stack:
            s=stack.pop()
            nodes_expanded+=1
            for nxt, c in neighbors(s):
                if nxt in seen: continue
                parents[nxt]=s
                seen.add(nxt)
                if nxt==GOAL:
                    path=reconstruct(parents,nxt)
                    return Result(True,len(path)-1,nodes_expanded,len(path)-1,time.perf_counter()-t0,path)
                stack.append(nxt)
        return Result(False, -1, nodes_expanded, -1, time.perf_counter()-t0, [])

    if algo in ("ucs","astar"):
        g=defaultdict(lambda: float("inf"))
        g[start]=0
        parents={}
        cnt=itertools.count()
        def h(s):
            if heuristic is None: return 0
            return heuristic(s)
        open_heap=[(g[start]+h(start), next(cnt), start)]
        closed=set()
        while open_heap:
            f,_,s=heapq.heappop(open_heap)
            if s in closed: continue
            closed.add(s)
            nodes_expanded+=1
            if s==GOAL:
                path=reconstruct(parents,s)
                return Result(True, g[s], nodes_expanded, len(path)-1, time.perf_counter()-t0, path)
            for nxt,c in neighbors(s):
                tentative=g[s]+c
                if tentative<g[nxt]:
                    g[nxt]=tentative
                    parents[nxt]=s
                    heapq.heappush(open_heap,(tentative+(0 if algo=="ucs" else h(nxt)), next(cnt), nxt))
        return Result(False, -1, nodes_expanded, -1, time.perf_counter()-t0, [])

    raise ValueError("Unknown algo")

# ---------- Heuristics ----------
goal_pos={v:i for i,v in enumerate(GOAL)}

def manhattan(s):
    dist=0
    for idx,val in enumerate(s):
        if val==0: continue
        gi=goal_pos[val]
        r1,c1=divmod(idx,3)
        r2,c2=divmod(gi,3)
        dist+=abs(r1-r2)+abs(c1-c2)
    return dist

def linear_conflict(s):
    dist=manhattan(s)
    for r in range(3):
        row=s[3*r:3*r+3]
        goal_row=[1+3*r,2+3*r,3+3*r]
        tiles=[x for x in row if x in goal_row]
        for i in range(len(tiles)):
            for j in range(i+1,len(tiles)):
                if goal_row.index(tiles[i])>goal_row.index(tiles[j]):
                    dist+=2
    for c in range(3):
        col=s[c::3]
        goal_col=[c+1,c+4,c+7]
        tiles=[x for x in col if x in goal_col]
        for i in range(len(tiles)):
            for j in range(i+1,len(tiles)):
                if goal_col.index(tiles[i])>goal_col.index(tiles[j]):
                    dist+=2
    return dist

# ---------- Experiments ----------
def run_experiments(num=5, seed=7):
    random.seed(seed)
    starts=[]
    while len(starts)<num:
        s=random_start()
        if is_solvable(s) and s not in starts:
            starts.append(s)
    rows=[]
    for i,s in enumerate(starts,1):
        for algo in ["bfs","dfs","ucs"]:
            res=run_search(s, algo=algo)
            rows.append(("Case"+str(i), algo.upper(), res.path_cost, res.nodes_expanded, res.secs))
        for name,h in [("A*-Manhattan", manhattan), ("A*-LinearConflict", linear_conflict)]:
            res=run_search(s, algo="astar", heuristic=h)
            rows.append(("Case"+str(i), name, res.path_cost, res.nodes_expanded, res.secs))
    return rows

if __name__=="__main__":
    from pprint import pprint
    rows=run_experiments()
    print("Case, Method, PathCost, NodesExpanded, Time(s)")
    for r in rows: print(",".join([str(x) for x in r]))

    import statistics as st
    def agg(label):
        subset=[r for r in rows if r[1]==label]
        return st.mean([r[3] for r in subset]), st.mean([r[4] for r in subset])
    u_nodes,u_time=agg("UCS")
    m_nodes,m_time=agg("A*-Manhattan")
    l_nodes,l_time=agg("A*-LinearConflict")
    print("\nAverages over 5 starts:")
    print(f"UCS  : nodes={u_nodes:.0f}, time={u_time:.4f}s")
    print(f"A* M : nodes={m_nodes:.0f}, time={m_time:.4f}s")
    print(f"A* LC: nodes={l_nodes:.0f}, time={l_time:.4f}s")

    print("\nHeuristic notes:")
    print("- Manhattan is admissible & consistent for 8-puzzle (each move shifts a tile by 1 Manhattan unit at most).")
    print("- Linear-Conflict = Manhattan + 2 per pair in row/col with reversed goal order; still admissible & consistent.")


Case, Method, PathCost, NodesExpanded, Time(s)
Case1,BFS,10,371,0.0008268100000350387
Case1,DFS,20688,23224,0.056439164000039455
Case1,UCS,10,601,0.0020093890000225656
Case1,A*-Manhattan,10,26,0.0002414979999798561
Case1,A*-LinearConflict,10,23,0.0004607650000139074
Case2,BFS,4,13,3.220099995360215e-05
Case2,DFS,214,232,0.00042870800001537646
Case2,UCS,4,27,7.963100000552004e-05
Case2,A*-Manhattan,4,5,5.136199996513824e-05
Case2,A*-LinearConflict,4,5,0.00010805700003402308
Case3,BFS,20,34664,0.06939189100000931
Case3,DFS,22492,25224,0.056292773999985
Case3,UCS,20,50118,0.23093945400000848
Case3,A*-Manhattan,20,756,0.006156717999999728
Case3,A*-LinearConflict,20,437,0.007098781999957282
Case4,BFS,17,9418,0.017146479000018644
Case4,DFS,47999,54877,0.17399594500000148
Case4,UCS,17,14346,0.06324744500000179
Case4,A*-Manhattan,17,80,0.000637245999996594
Case4,A*-LinearConflict,17,63,0.0010475700000256438
Case5,BFS,4,14,8.086000002549554e-05
Case5,DFS,46136,52626,0.12978320700000268
Case5,UC